In [1]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [1]:
import numpy as np
import matplotlib.pyplot as plt
c
from sklearn.datasets import fetch_openml
from numpy.linalg import det, inv

In [2]:
# Load the MNIST dataset, False makes it return the data as a NumPy array
mnist = fetch_openml('mnist_784', version=1, as_frame=False, parser='liac-arff')

# Flatten the images
X = mnist.data
y = mnist.target
#print(X.shape) #(70000, 784)

# Split the data
X_train, y_train = X[:60000], y[:60000]
X_test, y_test = X[60000:], y[60000:]

# Convert labels to integers
y_train = y_train.astype(int)
y_test = y_test.astype(int)

# Reduce dimensionality using PCA
pca = PCA(n_components=5)
X_train_pca = pca.fit_transform(X_train) #(50000, 5)
X_test_pca = pca.transform(X_test) #(10000, 5)

In [5]:
print(X_train_pca.shape)

(60000, 5)


Q1:


The S_j is in the S_k matrix, which is a diagonal matrix, the number of j is as same as the number of k. S_j is the value of j-th diagonal value of S_k. Therefore, we can compute the covariance as a vector. In line 12, S_k change the sigma and therefore r_ik will change with each iteration.   

Extra question:

Density estimation is valuable for understanding data distributions, but not a good choice for an intermediate step for classification.For harder classification problems like indoor scene classification, it's better to directly use classification methods without using intermediate density estimation step. This avoids unnecessary complexity and overfitting.

In [55]:
# class cluster_parameters:
#     def __init__(self, d):
#         self.mu = np.zeros(d)  # Initialize mean vector with zeros
#         self.sigma = np.ones(d)  # Initialize diagonal covariance vector with ones
#         self.pi = 1.0  # Initialize mixing coefficient

class cluster_parameters:
    def __init__(self, X, k):
        print(np.shape(X))
        self.N = np.shape(X)[0]
        self.D = np.shape(X)[1]# Number of data points and dimensions
        self.k = k  # Number of clusters

        # Initialize mean vectors with random samples from the data

        self.mu = np.mean(X,axis = 1).flatten()
        self.pi = np.shape(X)[0]/self.N
        self.sigma =np.diagonal(np.cov(X.T))
          # Initialize mixing coefficients with uniform distribution


def estimate_r_k(X, mu, sigma, pi):
    # Compute responsibilities for each data point
    # n datapoints length
    # k number of cluster
    # d demension
    n, d = X.shape
    k = len(pi)
    r_k = np.zeros((n,k))
    det_sigma = np.zeros(k)
    inv_sigma = np.zeros(k)
    for j in range(k):

      det_sigma[j] = np.prod(sigma) # Compute the determinant of the covariance matrix
      inv_sigma[j] = 1/sigma[j] # Compute the inverse of the covariance matrix

    print("r_k in est shape",r_k.shape)
    print("r_k in est",r_k)
    print("det_sigma in est",det_sigma.shape)
    print("inv_sigma in est",inv_sigma.shape)
    for i in range(n):
      for j in range(k):

        x_i = X[i]
        diff = x_i - mu[k]
        # r_k[i] = cluster_parameters_k.pi * (det_sigma ** -0.5) * np.exp(-0.5 * np.sum(diff ** 2 * inv_sigma))
        r_k[i,j] = pi[k] * ((2*np.pi)**5*det_sigma[j] )** -0.5 * np.exp(np.sum(-0.5 * diff.T**2 @ inv_sigma[j]),axis = 1)
        # r_k[i] = cluster_parameters_k.pi * np.prod(np.exp(-0.5 * (x_i - cluster_parameters_k.mu) ** 2 / cluster_parameters_k.sigma))
    r_k_sum = np.sum(r_k,axis = 1)
    for j in range(k):
      r_k[:,j]= r_k[:,j]/r_k_sum


    return r_k

def estimate_cluster_parameters(X, r_k, k):
    # Compute the updated parameters for each cluster
    # kth ckuster

    n, d = X.shape

    mu = np.sum(r_k[:,k].reshape(-1,1) * X, axis=0) / np.sum(r_k,axis = 0)[k]

    sigma = np.sum(r_k[:,k].reshape(-1,1) * (X - mu) ** 2, axis=0) / np.sum(r_k,axis = 0)[k] - mu ** 2

    pi = np.sum(r_k) / n

    return mu, sigma, pi


def dGMM(X, k, epoch=1, tol=1e-5):
    #Initialization
    n, d = X.shape
    #cluster_assignments = np.random.randint(0, k, size=n)  # Randomly assign clusters initially
    #print("cluster_assignments",cluster_assignments.shape) #(60000,)
    random_indices = np.random.randint(k,size=n)
    print('----')
    print(n)


    clust = [] #List to store the instances
    for j in range(k):
        clust.append(cluster_parameters(X[random_indices==j], j))

    #cluster_parameters_k = [cluster_parameters(X,k) for _ in range(k)]  # Initialize parameters for K clusters
    #print("cluster_parameters_k",cluster_parameters_k)
    r_k_clusters = np.ones((n, k))



    for iter in range(epoch):
        # E-step: Update responsibilities
        #for j in range(k):
            #r_k_clusters[:, j] = estimate_r_k(X, cluster_parameters_k[j])
        for k, cluster in enumerate(clust):

            print(f"Cluster {k+1}:")
            print(f"Means: {cluster.mu}")
            print(f"Variances: {cluster.sigma}")
            print(cluster.sigma.shape)
            print(f"Mixing Coefficients: {cluster.pi}")

            r_k_clusters[:, k] = estimate_r_k(X, cluster.mu, cluster.sigma, cluster.pi)#######

            print("rik: ", r_k_clusters)
            print("rik shape: ", r_k_clusters.shape)

        # M-step: Update cluster parameters

            mu, sigma, pi = estimate_cluster_parameters(X, r_k_clusters, k)
            cluster_parameters_k[j].mu = mu
            cluster_parameters_k[j].sigma = sigma
            cluster_parameters_k[j].pi = pi

        # Compute negative log-likelihood
        ll_iter = -np.sum(np.log(np.sum(r_k_clusters, axis=1)))

        if iter > 0 and np.abs(ll_iter - ll_prev) <= tol * np.abs(ll_iter):
            break

        ll_prev = ll_iter

    return cluster_assignments, cluster_parameters_k


In [56]:
k_clusters = 5
data_matrix = X_test_pca  # Use PCA-reduced data
print(np.shape(data_matrix))
assignments, parameters = dGMM(data_matrix, k_clusters) # RUN MODEL

# Access the parameters for each cluster:
for i, param in enumerate(parameters):
    print(f"Cluster {i + 1} - Mean: {param.mu}, Covariance Diagonal: {param.sigma}, Mixing Coefficient: {param.pi}")



(10000, 5)
----
10000
(2033, 5)
(2049, 5)
(1935, 5)
(1944, 5)
(2039, 5)
Cluster 1:
Means: [-147.98196037 -316.93639378  218.77718867 ... -274.07658233  187.71642277
  -36.05449962]
Variances: [337746.15186329 247776.45501992 206451.49436211 181382.39550615
 168088.64987523]
(5,)
Mixing Coefficients: 1.0


TypeError: object of type 'float' has no len()